In [5]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt

import torch
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric import nn as gnn

from data_load import graph_list, node_counter, assigner
from graph_vae import GraphVAE

In [6]:
f = h5.File("quark-gluon_data-set_n139306.hdf5")

In [4]:
# f = h5.File("quark-gluon_data-set_n139306.hdf5")
x, m0, pt, y = (torch.from_numpy(f['X_jets'][:5000]), torch.from_numpy(f['m0'][:5000]), 
                torch.from_numpy(f['pt'][:5000]), torch.from_numpy(f['y'][:5000]))
f.close()

In [11]:
f['y'][[4, 6, 7, 8]]

array([1., 0., 1., 1.], dtype=float32)

In [ ]:
class 

In [9]:
x.shape, m0.shape
# Later: different batch size and/or load full dataset

(torch.Size([5000, 125, 125, 3]), torch.Size([5000]))

In [21]:
torch.from_

array([0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
      dtype=float32)

In [6]:
graphs = graph_list(x)
counts = node_counter(graphs)
lengs = torch.LongTensor(np.hstack(assigner(counts)))

compress = torch_geometric.data.Batch.from_data_list(graphs)
G = compress.x.clone() # All nodes of all graphs cat together
E = compress.edge_index.clone()

In [7]:
from torch_geometric.utils import to_dense_batch, to_dense_adj

In [8]:
X, mask = to_dense_batch(G, lengs, fill_value=0, max_num_nodes=1000)
A = to_dense_adj(E, lengs, max_num_nodes=1000) # (batch, 1000, 1000)


In [9]:
X = X
A = A
X.shape, A.shape

(torch.Size([2, 1000, 3]), torch.Size([2, 1000, 1000]))

In [12]:
net = GraphVAE(3, 16, 3)
net

GraphVAE(
  (sage): ModuleList(
    (0): DenseSAGEConv(3, 16)
    (1): DenseSAGEConv(16, 16)
    (2): DenseSAGEConv(16, 3)
  )
  (drop): ModuleList(
    (0): Dropout(p=0.5, inplace=False)
    (1): Dropout(p=0.4, inplace=False)
    (2): Dropout(p=0.3, inplace=False)
  )
  (batch_norm): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool): ModuleList(
    (0): MinCut_Pool(
      (linear): Linear(in_features=16, out_features=500, bias=True)
    )
    (1): MinCut_Pool(
      (linear): Linear(in_features=16, out_features=250, bias=True)
    )
  )
  (tr_mu): Linear(in_features=3, out_features=16, bias=True)
  (tr_var): Linear(in_features=3, out_features=16, bias=True)
  (tr_rev): Linear(in_features=16, out_features=3, bias=True)
  (revsage): ModuleList(

In [42]:
Z, A, mu, logvar, L1, L2 = net(X, A)

In [43]:
Z.shape, A.shape

(torch.Size([2, 1000, 3]), torch.Size([2, 1000, 1000]))